# Preliminary operations

In [1]:
ls

 Il volume nell'unit� C � Windows
 Numero di serie del volume: BEF8-AC01

 Directory di c:\Users\Simone\Documents\Desktop\public_and_social_services

15/06/2023  15:56    <DIR>          .
15/06/2023  15:56    <DIR>          ..
06/06/2023  16:46             6.148 .DS_Store
13/06/2023  17:13    <DIR>          data
15/06/2023  16:20            19.137 Demo.ipynb
13/06/2023  17:13           174.889 EDA_domains.ipynb
15/06/2023  15:55         2.976.082 Extra_tools.ipynb
08/06/2023  17:08    <DIR>          flagged
15/06/2023  16:22            45.972 Langchain.ipynb
08/05/2023  15:03    <DIR>          ricette
               5 File      3.222.228 byte
               5 Directory  28.254.208.000 byte disponibili


In [2]:
cd C:\Users\Simone\Documents\Desktop\public_and_social_services

c:\Users\Simone\Documents\Desktop\public_and_social_services


# Notebook of Extra tool

In [3]:
%run Extra_tools.ipynb

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Simone\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Codice prestazione ambulatoriale  331 non-null    object
 1   Codice branca regionale           331 non-null    object
 2   Branca regionale                  331 non-null    object
 3   Tipo di prestazione               331 non-null    object
 4   Nome struttura                    331 non-null    object
 5   Comune struttura                  331 non-null    object
 6   Provincia struttura               331 non-null    object
 7   Indirizzo                         331 non-null    object
 8   Descrizione ASL                   331 non-null    object
 9   Codice struttura                  331 non-null    object
 10  Tipo struttura                    331 non-null    object
 11  Telefono                          331 non-null    object
 12  Fax                   

# Notebook of Langchain

In [6]:
cd C:\Users\Simone\Documents\Desktop\public_and_social_services

C:\Users\Simone\Documents\Desktop\public_and_social_services


In [7]:
%run Langchain.ipynb

Istruzione: Usa massimo 2 - 3 frasi per rispondere a domande relative alla sanità.
Contesto: Sei un agente AI per fornire informazioni generali all'utente legati all'ambito sanitario. Ricorda che non puoi effettuare prenotazioni o azioni simili.
Domanda: Mi dai una definizione di calcitonina?



> Entering new AgentExecutor chain...
 La calcitonina è un ormone peptidico prodotto dalle cellule C delle ghiandole paratiroidi.
Action: Wikipedia Search
Action Input: Calcitonina
Observation: Page: Calcitonina
Summary: La calcitonina è un ormone costituito da un polipeptide di 32 aminoacidi che viene prodotto, negli esseri umani, dalle cellule parafollicolari della tiroide (note anche come cellule C, da clear, chiaro, per la loro scarsa colorabilità al microscopio ottico rispetto ai tireociti), dalle cellule K dell'epitelio respiratorio e in molti altri animali dall'Ultimopharyngeal body (ghiandola associata all'ultimo arco branchiale).

La principale funzione della calcitonina è l'abbassamen

# Demo

Per accedere al chatbot: http://127.0.0.1:7860

Per utilizzare il tool_extra per trovare gli ambulatori utilizzare la seguente funzione: match_recipe_to_dataset_and_create_df

In [8]:
cd C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette

C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


In [15]:
Varicella = match_recipe_to_dataset_and_create_df('ricetta_4.pdf',dataset)

In [13]:
filter_data(Varicella,'BERGAMO','Sì')

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
100,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,CENTRO BIOMEDICO BERGAMASCO SRL,BERGAMO,BG,ROTONDA DEI MILLE 3/A,ATS DI BERGAMO,1359.0,Servizio di Medicina di Laboratorio,035222332,035236267,Sì,7.75
195,VIRUS VARICELLA ZOSTER ANTICORPI (E.I.A.),91271,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,LABORATORIO BIOANALITICO DELTA SRL,BERGAMO,BG,VIA CLARA MAFFEI 14,ATS DI BERGAMO,1252.0,Servizio di Medicina di Laboratorio,035221565,035235848,Sì,7.75
